<a href="https://colab.research.google.com/github/gbr-git/Masters_Project/blob/main/Customer_Feedback_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install kaggle
! pip install -q kaggle

 # Download kaggle.json file from your kaggle profile and upload
from google.colab import files
files.upload()

# Download yelp dataset from kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d yelp-dataset/yelp-dataset

# UnZip the downloaded data
import zipfile
with zipfile.ZipFile('/content/yelp-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('')

# Remove the ZIP file
import os
os.remove("/content/yelp-dataset.zip")

# Save data to drive
from google.colab import drive
drive.mount('/content/drive')

Saving kaggle.json to kaggle (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 4.07G/4.07G [00:40<00:00, 68.5MB/s]
100% 4.07G/4.07G [00:40<00:00, 107MB/s] 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pyspark
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName('yelp_dataset').getOrCreate()

In [3]:
reviews_sk = spark.read.json('/content/yelp_academic_dataset_review.json')
# We will get a sample of 10L from the dataset
sample_review=reviews_sk.sample(False, 0.1, seed=0).limit(1000000)
reviews_pd = sample_review.select("*").toPandas()

# Overview of the converted sample
reviews_pd.to_csv('yelp_review_sample.csv')
reviews_pd.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,YjUWPpI6HXG530lwP-fb2A,0,2014-02-05 20:30:30,0,saUsX_uimxRlCVr67Z4Jig,3.0,Family diner. Had the buffet. Eclectic assortm...,0,8g_iMtfSiwikVnbP2etR0A
1,gebiRewfieSdtt17PTW6Zg,0,2016-07-25 07:31:06,0,pUycOfUwM8vqX7KjRRhUEA,3.0,Had a party of 6 here for hibachi. Our waitres...,0,59MxRhNVhU9MYndMkz0wtw
2,YtSqYv1Q_pOltsVPSx54SA,0,2013-06-24 11:21:25,0,oyaMhzBSwfGgemSGuZCdwQ,5.0,Tremendous service (Big shout out to Douglas) ...,0,Dd1jQj7S-BFGqRbApFzCFw
3,x4XdNhp0Xn8lOivzc77J-g,0,2013-08-15 15:27:51,0,G_5UczbCBJriUAbxz3J7Tw,5.0,Best thai food in the area. Everything was au...,0,clWLI5OZP2ad25ugMVI8gg
4,j8JOZvfeHEfUWq3gEz6ABQ,0,2014-06-11 14:55:14,3,Ki90-hauSIfW_7sBuBBqfA,2.0,NEVER AGAIN. This is a so called restaurant th...,2,Z2cOL3n9V8NoguJ-uU_Nvw


In [4]:
!cp 'yelp_review_sample.csv' '/content/drive/MyDrive'

# !cp 'yelp_business.csv' '/content/drive/MyDrive'

In [ ]:
business_sk=spark.read.json('/content/yelp_academic_dataset_business.json')
business_pd=business_sk.select("*").toPandas()

In [ ]:
business_pd.columns

In [ ]:
# If we look at the categories, all most every store selling food has one of these tags
food_tags=['Food','Restaurants','Cafes']

# Save the dataset
business_pd.to_csv('yelp_business.csv')

In [ ]:
# We will add a column serves_food to identify if the store serves food or not
serves_food=[]
for i in range(len(business_pd)):
    try:
        if(business_pd.iloc[i]['categories'] is not None):
            categories=business_pd.iloc[i]['categories'].split(',')
            categories=[en.replace(' ','') for en in categories]
            matched_tags=(list(set(categories).intersection(food_tags)))
            if(len(matched_tags)>0):
                serves_food.append(1)
            else:
                serves_food.append(0)
        else:
            serves_food.append(0)
    except:
        print(business_pd.iloc[i]['categories'])
        serves_food.append(2)

business_pd.head()

In [ ]:
business_pd['serves_food']=serves_food
business_food=business_pd[business_pd['serves_food']==1]
business_food.to_csv('yelp_business_food.csv')

In [ ]:
# Copy the dataset to drive
!cp 'yelp_business_food.csv' '/content/drive/MyDrive'

In [ ]:
# ids of the food stores
business_food_ids=business_food['business_id']


# From the review dataset we will extract the reviews for food stores
business_food_reviews=reviews_pd[reviews_pd['business_id'].isin(business_food_ids)]
business_food_reviews.to_csv('yelp_business_food_reviews.csv')
business_food_reviews.shape


!cp 'yelp_business_food_reviews.csv' '/content/drive/MyDrive'


# Food store Reviews
business_food_reviews.head()

In [ ]:
# Overview of the dataset
print('Number of unique Reviews:',str(len(business_food_reviews)))
print('Number of unique Businesses:',str(len(business_food_reviews.business_id.unique())))
print('Number of unique Users that posted a Review:',str(len(business_food_reviews.user_id.unique())))

In [ ]:
# Merging the store info with review  dataset
review_business_data_merged=pd.merge(business_food_reviews,business_food,how='left',on='business_id')
review_business_data_merged.head()

In [ ]:
# Copy to dive
review_business_data_merged.to_csv('yelp_reviews_business_merged.csv')
!cp 'yelp_reviews_business_merged.csv' '/content/drive/MyDrive'

In [ ]:
# Drop unwanted columns
# review_business_data_merged=review_business_data_merged.drop(columns=['hours','attributes','useful','funny'])
review_business_data_merged.head()

In [ ]:
review_business_data_merged=pd.read_csv('/content/drive/MyDrive/yelp_reviews_business_merged.csv')
review_business_data_merged=review_business_data_merged.drop(columns=['Unnamed: 0'])

# CREATING A NEW COLUMN IN THE DATASET FOR THE NUMBER OF WORDS IN THE REVIEW
review_business_data_merged['length(No of words count in Text)'] = review_business_data_merged['text'].apply(len)
review_business_data_merged.head()
# SHAPE OF THE DATASET
print("Shape of the dataset:")
print(review_business_data_merged.shape)
# COLUMN NAMES
print("Column names:")
print(review_business_data_merged.columns)
# DATATYPE OF EACH COLUMN
print("Datatype of each column:")
print(review_business_data_merged.dtypes)
# DATASET SUMMARY
review_business_data_merged[['cool', 'funny', 'stars_x', 'useful', 'latitude', 'longitude', 'review_count', 'length(No of words count in Text)']].describe()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# COMPARING TEXT LENGTH TO STARS
graph = sns.FacetGrid(data=review_business_data_merged, col='stars_x')
graph.map(plt.hist, 'length(No of words count in Text)', bins=50, color='green')

In [ ]:
# GETTING THE MEAN VALUES OF THE VOTE COLUMNS WRT THE STARS ON THE REVIEW
stval = review_business_data_merged.groupby('stars_x').mean()
stval
# FINDING THE CORRELATION BETWEEN THE VOTE COLUMNS
stval.corr()
stval[['cool', 'funny', 'useful', 'length(No of words count in Text)']].corr()

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x=review_business_data_merged['stars_x'],y=review_business_data_merged['length(No of words count in Text)'])

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x=review_business_data_merged['stars_x'])

In [ ]:
review_business_data_merged.head()

In [ ]:
review_business_data_merged['cool'].plot(kind='box', title='Voting Columns(cool)')
plt.show()
review_business_data_merged['funny'].plot(kind='box', title='Voting Columns(funny)')
plt.show()
review_business_data_merged['useful'].plot(kind='box', title='Voting Columns(useful)')
plt.show()
review_business_data_merged['review_count'].plot(kind='box', title='Voting Columns(useful)')
plt.show()

In [ ]:
plt.figure(figsize=(35,15))
sns.boxplot(x=review_business_data_merged['cool'],y=review_business_data_merged['length(No of words count in Text)'])

In [ ]:
plt.figure(figsize=(35,15))
sns.boxplot(x=review_business_data_merged['funny'],y=review_business_data_merged['length(No of words count in Text)'])

In [ ]:
plt.figure(figsize=(35,15))
sns.boxplot(x=review_business_data_merged['useful'],y=review_business_data_merged['length(No of words count in Text)'])

In [ ]:
# Box Plot
review_business_data_merged['stars_x'].plot(kind='box', title='Review Ratings')
plt.show()

In [ ]:
! pip install ptitprince
import ptitprince as pt
rp = pt.RainCloud(x=review_business_data_merged['stars_x'],y=review_business_data_merged['length(No of words count in Text)'], data=review_business_data_merged, width_viol=.8, width_box=.4, orient='h')

In [ ]:
category_counts = business_pd['categories'].str.split(';').explode().value_counts()
# Get the top N categories
top_n = 25
top_categories = category_counts.head(top_n)

# Display the top performing categories
print("Top Performing Categories:")
print(top_categories)
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
top_categories.plot(kind='bar', color='skyblue')
plt.title(f"Top {top_n} Performing Categories")
plt.xlabel("Category")
plt.ylabel("Count")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
#Get the distribution of the ratings
color = sns.color_palette()
x=business_pd['city'].value_counts()
x=x.sort_values(ascending=False)
x=x.iloc[0:20]
plt.figure(figsize=(16,4))
ax = sns.barplot(x.index, x.values, alpha=0.8,color=color[3])
plt.title("Which city has the most reviews?")
locs, labels = plt.xticks()
plt.setp(labels, rotation=45)
plt.ylabel('# businesses', fontsize=12)
plt.xlabel('City', fontsize=12)

#adding the text labels
rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()

In [ ]:
user_agg=reviews_pd.groupby('user_id').agg({'review_id':['count'],'date':['min','max'],
                                'useful':['sum'],'funny':['sum'],'cool':['sum'],
                               'stars':['mean']})
user_agg=user_agg.sort_values([('review_id','count')],ascending=False)
print("          Top 10 Users in Yelp")
user_agg.head(10)

In [ ]:
business_pd['name'] = business_pd['business_id'].map(df_yelp_business_restaurants.set_index('business_id')['name'])
# take the top only 20 most occurances restaurants for analysis. Because the dataset has over thousand business id. It will be really inefficient if we take all of the business name
top_restaurants = business_pd.name.value_counts().index[:20].tolist()
# create a new dataframe called 'df_review_top' that only contains the reviews of 'top_restaurants'
df_review_top = business_pd.loc[business_pd['name'].isin(top_restaurants)]
# now we take the name column and count mean of stars of each unique business name (I am using pandas groupby function for this. If you don't know what pandas groupby function does, please search on Google)
# then sort values in ascending order
# then plot the graph
df_review_top.groupby(df_review_top.name)['stars_x'].mean().sort_values(ascending=True).plot(kind='barh',figsize=(12, 10))
# set y ticks font size to 18 for easy readable purposes.
plt.yticks(fontsize=18)
# set the title of the graph
plt.title('Top rated restaurants on Yelp',fontsize=20)
# set y label of the graph and change fontsize to 18 for readablity purposes.
plt.ylabel('Restaurants names', fontsize=18)
# set x label to 'Ratings'
plt.xlabel('Ratings', fontsize=18)
# now finally show the graph in a new window.
plt.show()